In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)""

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# Any results you write to the current directory are saved as output.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataframe = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
print(dataframe.shape)
dataframe.head()

In [ ]:
dataframe.emotion.unique()

In [ ]:
emotion_mapping = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
dataframe.emotion.value_counts()

In [ ]:
plt.hist(dataframe.emotion,bins=7)
plt.show()

# Changing the size to make it equal.. 

In [ ]:
import tensorflow as tf
SIZE=48
def augment_pixels(string_grayscale):
    image = np.array(string_grayscale.split(' ')).reshape(SIZE, SIZE).astype('float32')
    image = tf.image.random_flip_left_right(image.reshape(SIZE,SIZE,1))# Pad image size
    image = tf.image.resize_with_crop_or_pad(image, SIZE + 12, SIZE + 12) # Random crop back to the original size
    image = tf.image.random_crop(image, size=[SIZE, SIZE, 1])
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.clip_by_value(image, 0, 255)
    augmented = image.numpy().reshape(SIZE,SIZE)
    str_augmented = ' '.join(augmented.reshape(SIZE*SIZE).astype('int').astype(str))
    return str_augmented

number_of_samples = dataframe.emotion.value_counts()
difference_between_samples = number_of_samples[number_of_samples.idxmax()] - number_of_samples
for emotion_id,diff in difference_between_samples.iteritems():
    sampled = dataframe.query("emotion==@emotion_id").sample(diff, replace=True)
    sampled['pixels'] = sampled.pixels.apply(augment_pixels)
    dataframe = pd.concat([dataframe, sampled])
    print(emotion_id, diff)

In [ ]:
dataframe.Usage.unique()

In [ ]:
# Check again to see if dataset size is similar across emotions.
plt.hist(dataframe.emotion,bins=7)
plt.show()

In [ ]:
fig = plt.figure(1)


for label in range(7):
    for j in range(3):
        px = dataframe[dataframe.emotion==label].pixels.iloc[j]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')
        ax = plt.subplot(3,3,j+1)
        ax.imshow(px)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotion_mapping[label])
        plt.tight_layout()
    plt.show()

In [ ]:
INTERESTED_LABELS = [0, 1, 2, 3, 4, 5, 6]

In [ ]:
dataframe = dataframe[dataframe.emotion.isin(INTERESTED_LABELS)]
dataframe.shape

## Preprocessing for neural network.`

In [ ]:
img_array = dataframe.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)

In [ ]:
img_array.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
le = LabelEncoder()
img_labels = le.fit_transform(dataframe.emotion)
img_labels = to_categorical(img_labels)
img_labels.shape

## Splitting the data into training and testing set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
X_train.dtype,X_valid.dtype

In [ ]:
# Normalizing results, as neural networks are very sensitive to unnormalized data.
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization
def build_net():

    net = Sequential()

    net.add(Conv2D(64,(5,5),input_shape=(img_width, img_height, img_depth),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    net.add(Conv2D(64,(5,5),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    
    net.add(MaxPooling2D(pool_size=(2,2)))
    net.add(Dropout(0.4))

    net.add(Conv2D(128,(3,3),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    net.add(Conv2D(128,(3,3),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    
    net.add(MaxPooling2D(pool_size=(2,2)))
    net.add(Dropout(0.4))

    net.add(Conv2D(256,(3,3),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    net.add(Conv2D(256,(3,3),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    
    net.add(MaxPooling2D(pool_size=(2,2)))
    net.add(Dropout(0.5))
    
    net.add(Conv2D(512,(3,3),activation='elu',padding='same',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    net.add(MaxPooling2D(pool_size=(2,2)))
    net.add(Dropout(0.5))

    net.add(Flatten())
        
    net.add(Dense(128,activation='elu',kernel_initializer="he_normal"))
    net.add(BatchNormalization())
    
    net.add(Dropout(0.6))
    
    net.add(Dense(num_classes,activation='softmax',kernel_initializer="he_normal"))
    
    net.compile(
        loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy']
    )
    
    net.summary()
    
    return net

In [ ]:

from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
# As the data in hand is less as compared to the task so ImageDataGenerator is good to go.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

In [ ]:

model = build_net() 



In [ ]:
batch_size = 32 #batch size of 32 performs the best.
epochs = 150
history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size), validation_data=(X_valid, y_valid),steps_per_epoch=len(X_train) / batch_size,
                              epochs=epochs,
                              callbacks=callbacks,use_multiprocessing=True
)
model.save("model.h5")

In [ ]:
import seaborn as sns
from matplotlib import pyplot 
sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='valid')
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='valid')
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('epoch_history_dcnn.png')
pyplot.show()


In [ ]:
import scikitplot
from sklearn.metrics import classification_report
yhat_valid = model.predict_classes(X_valid)
scikitplot.metrics.plot_confusion_matrix(np.argmax(y_valid, axis=1), yhat_valid, figsize=(7,7))
pyplot.savefig("confusion_matrix_dcnn.png")
